# Setup

In [1]:
import numpy as np
import open3d as o3d
from sklearn import cluster
import pandas as pd

from PointCloudDataFrame import PointCloudDataFrame

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
filename = 'blake-new.ply'
pcd_raw = o3d.io.read_point_cloud(f'input/{filename}')
df_raw = PointCloudDataFrame.from_pcd(pcd_raw)

# o3d.visualization.draw_geometries([pcd_raw.voxel_down_sample(voxel_size=0.001)])

In [18]:
df_scaled = df_raw.copy()
df_scaled[['x', 'y', 'z']] *= 0.75
pcd_scaled = df_scaled.to_pcd()

In [19]:
pcd = pcd_scaled.voxel_down_sample(voxel_size=0.001)  # 1mm
df = PointCloudDataFrame.from_pcd(pcd)

o3d.visualization.draw_geometries([pcd])

In [20]:
o3d.io.write_point_cloud(f'input/{filename}', pcd_scaled)

True

In [6]:
df_filter1 = df[(df['s'] < 0.125) & (df['v'] > 0.2)]
pcd_filter1 = df_filter1.to_pcd()

In [7]:
_, ind = pcd_filter1.remove_radius_outlier(nb_points=70, radius=0.005)
pcd_inliers = pcd_filter1.select_by_index(ind)
df_inliers = PointCloudDataFrame.from_pcd(pcd_inliers)

In [8]:
model_kmeans = cluster.KMeans(n_clusters=120, n_init='auto')
model_kmeans.fit(pcd_inliers.points)

KMeans(n_clusters=120, n_init='auto')

model_dbscan = cluster.DBSCAN(eps=0.003, min_samples=10)
model_dbscan.fit(pcd_inliers.points)

labels = model_dbscan.labels_ 
num_labels = len(set(labels).difference({-1}))
print(num_labels)

centroids = []
for i in range(num_labels):
    centroids.append(
        np.mean(df_inliers[labels==i][['x', 'y', 'z']], axis=0)
    )

pcd_centroids = o3d.geometry.PointCloud()
pcd_centroids.points = o3d.utility.Vector3dVector(centroids)

o3d.visualization.draw_geometries([pcd_inliers, pcd_centroids])

In [9]:
labels = model_kmeans.labels_ 
num_labels = len(set(labels).difference({-1}))
print(num_labels)

model_kmeans_centers = []
for i in range(num_labels):
    model_kmeans_centers.append(
        np.mean(df_inliers[labels==i][['x', 'y', 'z']], axis=0)
    )

120


In [10]:
for i in range(num_labels):
    point = model_kmeans_centers[i]
    filter = ((df_scaled['x'] - point['x'])**2 + (df_scaled['y'] - point['y'])**2 + (df_scaled['z'] - point['z'])**2) < 0.006**2

    pcd_temp = df_scaled[filter].to_pcd()
    pcd_temp.paint_uniform_color([0, 0.8, 0])

    o3d.visualization.draw_geometries([pcd_temp, pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


KeyboardInterrupt: 

In [ ]:
label_names = ['', 'E23', 'E58', 'E21', 'E34', 'E93', '', 'E25', 'E53', 'E123', 
			'', 'E106', '', 'E36', 'E38', '', 'E13', 'E78', 'E7', '',
			'E46', 'E112', '', 'E121', 'E9', 'E11', 'E7', 'E68', 'E80', 'E86',
			'COM', '', 'E101', '', 'E107', 'E99', 'E1', 'E117', 'E22', '',
            'E24', '', '', 'E54', 'VREF', 'E108', '', 'E20', 'E12', 'E102',
			'E116', '', 'E103', 'E61', 'E17', 'E109', 'E10', 'E65', 'E44', 'E128', 
			'E48', 'E3', 'E49', 'E118', 'E27', 'E15', 'E55', 'E5', '', 'E87', 
			'E40', 'E62', 'E124', '', 'E63', 'E94', 'E6', 'E100', 'E79', 'E8', 
			'E14', 'E28', 'E4', 'E2', '', 'E31', 'E26', 'E37', 'E110', 'E33',
			'E113', '', 'E90', '', 'E56', 'E39', '', '', 'E120', 'E32',
			'', '', '', '', 'E144', 'E119', '', 'E43', '', '',
			'']

In [ ]:
total_labels = {'VREF', 'COM'}.union(f'E{i}' for i in range(1, 129))
missing_labels = total_labels.difference(label_names)
print(missing_labels)

{'E60', 'E81', 'E18', 'E29', 'E70', 'E92', 'E95', 'E84', 'E115', 'E47', 'E76', 'E98', 'E126', 'E52', 'E97', 'E88', 'E64', 'E71', 'E82', 'E104', 'E57', 'E96', 'E69', 'E74', 'E16', 'E125', 'E50', 'E51', 'E105', 'E91', 'E41', 'E72', 'E42', 'E19', 'VERF', 'E85', 'E30', 'E75', 'E122', 'E45', 'E111', 'E114', 'E59', 'E127', 'E66', 'E83', 'E77', 'E73', 'E35', 'E89', 'E67'}


In [ ]:
df_scaled['label'] = ''
for i, l in enumerate(label_names):
    if l:
        point = centroids[i]
        filter = ((df_scaled['x'] - point['x'])**2 + (df_scaled['y'] - point['y'])**2 + (df_scaled['z'] - point['z'])**2) < 0.006**2
        df_scaled.loc[filter, 'label'] = l
# df_scaled

In [ ]:
point = np.mean(
    pd.DataFrame([
        centroids[label_names.index('VREF')],
        # centroids[label_names.index('E52')],
    ]), axis=0
)
point['x'] -= 0.026
point['y'] -= 0.108
point['z'] -= 0.138
print(point.tolist())

filter = ((df_scaled['x'] - point['x'])**2 + (df_scaled['y'] - point['y'])**2 + (df_scaled['z'] - point['z'])**2) < 0.006**2

pcd_temp = df_scaled[filter].to_pcd()
pcd_temp.paint_uniform_color([0, 0.8, 0])

o3d.visualization.draw_geometries([pcd_temp, pcd])

[-0.03587809806510377, -0.15152675894395964, 0.09508902913996292]


In [ ]:
new_points = {
    'E67': pd.Series((-0.05901098614067173, -0.12931973568786664, 0.16823132620646106), index=('x', 'y', 'z')),
    'E60': pd.Series((-0.08201098614067173, -0.10431973568786665, 0.16823132620646106), index=('x', 'y', 'z')),
    'E72': pd.Series((-0.029878098065103772, -0.14052675894395963, 0.16008902913996292), index=('x', 'y', 'z')),
    'E75': pd.Series((-0.03587809806510377, -0.15152675894395964, 0.09508902913996292), index=('x', 'y', 'z')),
}

In [ ]:
for l, p in new_points.items():
    filter = ((df_scaled['x'] - p['x'])**2 + (df_scaled['y'] - p['y'])**2 + (df_scaled['z'] - p['z'])**2) < 0.006**2
    df_scaled.loc[filter, 'label'] = l
# df_scaled

In [ ]:
df_scaled.write_label(f'label/{filename.split(".")[0]}.txt')